In [1]:
from itertools import starmap, chain
from collections import Counter

import numpy as np
import ggplot
from fn import F

from chempred.chemdner import read_abstracts, read_annotations, pair, annotate_abstract
from chempred import preprocessing as pp
import prototype

/home/ilia/.venvs/py3/lib/python3.5/site-packages/ggplot/utils.py:81: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  pd.tslib.Timestamp,
/home/ilia/.venvs/py3/lib/python3.5/site-packages/ggplot/stats/smoothers.py:4: FutureWarning: The pandas.lib module is deprecated and will be removed in a future version. These are private functions and can be accessed from pandas._libs.lib instead
  from pandas.lib import Timestamp
/home/ilia/.venvs/py3/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
Using TensorFlow backend.


In [7]:
window = 5
max_sample_len = 800

def len_filter(sequence, maxlen=max_sample_len):
    return [obj for obj in sequence if len(obj) <= maxlen]


abstracts_train = read_abstracts("chemdner_corpus/training.abstracts.txt")
anno_train = read_annotations("chemdner_corpus/training.annotations.txt")
pairs_train = list(zip(abstracts_train, pair(abstracts_train, anno_train)))
annotated_abstracts_train = list(starmap(F(annotate_abstract, guided=True), pairs_train))

abstracts_test = read_abstracts("chemdner_corpus/evaluation.abstracts.txt")
anno_test = read_annotations("chemdner_corpus/evaluation.annotations.txt")
pairs_test = list(zip(abstracts_test, pair(abstracts_test, anno_test)))
annotated_abstracts_test = list(starmap(F(annotate_abstract, guided=True), pairs_test))

pos_cls = Counter(
    r[-1] for r in chain.from_iterable(anno for _, anno in anno_train + anno_test)
)

In [3]:
annotated_items = [anno[3] for anno in 
                   chain.from_iterable(items for _, items in anno_train + anno_test)]

In [7]:
print(max(annotated_items, key=lambda x: len(x.encode())))

6-(1,1,5-trimethyl-5α-hydroxycyclohexanyl)-6'-(1',1',5'-trimethyl-2'β-hydroxycyclohexanyl)-9,13,9',13'-tetramethyloctadec-7,9,11,13,15,7',9',11',13'-nonene-5α-D-arabinopyranosyl(2a → 1b)-β-D-arabinopyranosyl-(2b → 1c)-β-D-arabinopyranosyl-2'-β-D-arabinopyranosyl-(2d → 1e)-α-D-arabinopyranosyl-(2e → 1f)-α-D-arabinopyranoside


In [8]:
# TODO rename generate_training_samples
joined_samples_train, joined_cls_train = pp.join_tokens_in_samples(
    *pp.generate_training_samples(annotated_abstracts_train, pos_cls, window, False))
samples_train, cls_train = map(len_filter, [joined_samples_train, joined_cls_train])
joined_samples_test, joined_cls_test = pp.join_tokens_in_samples(
    *pp.generate_training_samples(annotated_abstracts_test, pos_cls, window, False))
samples_test, cls_test = map(len_filter, [joined_samples_test, joined_cls_test])
# max_sample_len = max(map(len, samples_train+samples_test))

padded_samples_train, padded_cls_train, masks_train = pp.pad(samples_train, cls_train, 
                                                             max_sample_len)
padded_samples_test, padded_cls_test, masks_test = pp.pad(samples_test, cls_test, 
                                                          max_sample_len)

# onehot_samples_train, onehot_cls_train = map(
#     pp.encode_one_hot, [padded_samples_train, padded_cls_train])
# masked_samples_train = pp.mask_array(onehot_samples_train, masks_train)
# masked_cls_train = pp.mask_array(onehot_cls_train, masks_train)

# onehot_samples_test, onehot_cls_test = map(
#     pp.encode_one_hot, [padded_samples_test, padded_cls_test])
# masked_samples_test = pp.mask_array(onehot_samples_test, masks_test)
# masked_cls_test = pp.mask_array(onehot_cls_test, masks_test)

onehot_cls_train = pp.encode_one_hot(padded_cls_train)
masked_cls_train = pp.mask_array(onehot_cls_train, masks_train)

onehot_cls_test = pp.encode_one_hot(padded_cls_test)
masked_cls_test = pp.mask_array(onehot_cls_test, masks_test)

In [20]:
padded_samples_train.shape

(143616, 500)

In [8]:
# class_counts = np.vstack([np.unique(cls, return_counts=True)[1] for cls in 
#                           (padded_cls[mask] for padded_cls, mask in zip(padded_classes, masks))])
# class_counts[:10]

array([[34, 11],
       [36, 11],
       [33, 11],
       [29, 11],
       [29, 11],
       [26,  9],
       [24,  9],
       [24,  9],
       [18,  9],
       [22,  9]])

In [9]:
len(joined_samples_train), padded_samples_train.shape, len(joined_samples_test), padded_samples_test.shape

(143616, (143616, 800), 123380, (123380, 800))

In [10]:
from keras import backend as k
from keras import losses
from keras import models
from keras import layers
from keras import callbacks

In [ ]:
maxlen = padded_samples_train.shape[1]
nchar = 256
ncls = 2
nrec = 2
batchsize = 200

l_in = layers.Input(shape=(maxlen,), name="l_in")
l_emb = layers.Embedding(nchar, 50, mask_zero=True, input_length=maxlen)(l_in)

# model = models.Model(l_in, l_emb)

# model.predict(padded_samples_train[:1]).shape

# # l_mask = layers.Masking(mask_value=0, name="l_mask")(l_in)
l_rec = prototype.build_rec([200, 200], [0, 0.1], [0, 0.1])(l_emb)
l_out = layers.TimeDistributed(
    layers.Dense(ncls, activation='softmax'), name="l_out")(l_rec)
model = models.Model(l_in, l_out)
model.compile(optimizer="Adam", loss="binary_crossentropy", metrics = ["accuracy"])

filepath = "models/emb-length-800/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = callbacks.ModelCheckpoint(filepath, monitor="val_acc", verbose=1, 
                                       save_best_only=True, mode="max")
# # reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3,min_lr=0.0001)

callbacks_ = [checkpoint]

model.fit(padded_samples_train, masked_cls_train, batch_size=batchsize, epochs=30, verbose=1,
          validation_data=(padded_samples_test, masked_cls_test), callbacks=callbacks_)

Epoch 1/30
 13000/143616 [=>............................] - ETA: 7319s - loss: 0.5395 - acc: 0.7239

In [43]:
print(test2_x_encoded[:1].argmin(axis=2))
print(net2.predict(test2_x_encoded[:1]).argmax(axis=2))

[[0 0 1 1 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
[[0 0 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]
